In [101]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import progressbar as progressbar

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
print(f'TF version: {tf.__version__}')
tf.config.list_physical_devices()

TF version: 2.1.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Read Data

In [103]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '../datasets/'

## Amazon Fashion

In [104]:
# data_path = 'data/Amazon/'
# file_name = 'Amazon_full' # file_name = 'Amazon_05_users' 
# file_name = 'Amazon_01_users'
# file_name = 'am_80k_users'
file_name = 'am_40k_users'

## MovieLens

In [85]:
data_path = 'data/ML/'
# file_name = 'ml_1m'
# file_name = 'ML_full' # file_name = 'ML_05_users'
file_name = 'ML_01_users'

In [105]:
df = pd.read_pickle(path + file_name)
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes
df.head()

,user,item,datetime,rating,user_id,item_id
1845392,AGYA6NBNU1VUH,B0020B4EYU,2015-04-14,3.0,34371,11002
919114,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
5872323,AGYA6NBNU1VUH,B00KR6I6IK,2015-04-14,5.0,34371,75683
1237107,AGYA6NBNU1VUH,B0000ZFDD4,2015-04-14,3.0,34371,179
9971633,AGYA6NBNU1VUH,B008GXBRNM,2015-12-20,5.0,34371,35964


### Create ML split (paper)

In [7]:
test_set = pd.DataFrame()
train_set = pd.DataFrame()
pbar = progressbar.ProgressBar()
for u in pbar(df.user_id.unique()):
    last_item = df[df.user_id==u].iloc[-1:]
    test_set = test_set.append(last_item)
    remaining = df[df.user_id==u].iloc[:-1]
    train_set = pd.concat([train_set, remaining])

100% |########################################################################|


In [87]:
test_set = pd.read_pickle(path + 'ml_1m_test')
train_set = pd.read_pickle(path + 'ml_1m_train')

### Subset data

In [36]:
from Data_prep import leave_users_out
remaining, subset = leave_users_out(df, 40000)
print(len(subset))
df = subset
df.user_id = df.user_id.astype('category').cat.codes
df.item_id = df.item_id.astype('category').cat.codes

351771


---
# Data Prep

## Dataset Params

In [106]:
val_perc = test_perc = 0.1
n_last_items_val = n_last_items_test = 1

total_items = len(df.item_id.unique())
total_users = len(df.user_id.unique())

## Create Split

In [107]:
from Data_prep import train_val_test_split
datasets = train_val_test_split(df, val_perc, test_perc, n_last_items_val, n_last_items_test)
train_set, val_set, test_set = datasets

---
# Neural Collaborative Filtering (NCF)
Using the NCF framework we build Generalized Matrix Factorisation (GMF), Multiplayer Perceptron Matrix Factorisation (MLP) and combine the two in Neural Matrix Factorisation (NeuMF)
- paper: http://papers.www2017.com.au.s3-website-ap-southeast-2.amazonaws.com/proceedings/p173.pdf
- blog: https://medium.com/@victorkohler/collaborative-filtering-using-deep-neural-networks-in-tensorflow-96e5d41a39a1
- code: https://github.com/Leavingseason/NeuralCF/blob/master

## Set Parameters

In [110]:
run_num = 'am_40k_nolf_8'
# run_num = 'ml_1m_nolf_8'

In [111]:
GMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'nolf': 8,
    'regs': [0.000001,0.000001], #ML: [0.000001,0.000001]
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/GMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [112]:
MLP_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.0001,0,0,0],   #ML: [+-0.0005,0,0,0],
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/MLP_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

In [113]:
NeuMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'layers': [16,32,16,8],
    'reg_layers': [0.0001,0,0,0],
    'reg_mf': [0.000001,0.000001],
    'nolf': 8,
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/NeuMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

## Init

In [114]:
from NCF import NCF
NCF = NCF(total_users, total_items, GMF_params, MLP_params, NeuMF_params)

NCF.build_GMF_model()
NCF.build_MLP_model()
NCF.build_NeuMF_model()

## Training

### Read Stored Samples (3min for ML_1m)

In [116]:
# sample_path = '../NeuMF_storage/samples_ml_1m/'
# sample_name = 'ml_1m_sample_'

sample_path = '../NeuMF_storage/samples_am_40k/'
sample_name = 'am_40k_sample_'

In [117]:
%%time
import csv
samples = []
pbar = progressbar.ProgressBar()
for sample_num in pbar(range(20)):
    with open(f'{sample_path}{sample_name}{sample_num}.csv', 'r') as read_obj:
        sample = [[], [], []]
        csv_reader = csv.reader(read_obj)
        sample_str = list(csv_reader)
        
        for user_input, item_input, label in zip(sample_str[0], sample_str[1], sample_str[2]):
            sample[0].append(int(user_input))
            sample[1].append(int(item_input))
            sample[2].append(int(label))
        
    samples.append(sample)

CPU times: user 37 s, sys: 3.22 s, total: 40.2 s
Wall time: 40.3 s


### Fit model

## GMF

In [119]:
GMF_val_metrics = NCF.train_model('GMF', samples, train_set, val_set.iloc[:800], store_path=f'../weights/GMF_weights_{run_num}/GMF_weights') 


Fitting GMF with parameters:
learning_rate                                             0.001
batch_size                                                  256
nolf                                                          8
regs                                             [1e-06, 1e-06]
epochs                                                       20
sample_size                                              343771
num_neg                                                       4
ckpt_dir         ../NeuMF_storage/GMF_ckpts_am_40k_nolf_8/ckpts
optimizer                                                  Adam
Name: 0, dtype: object
Epoch: 0
Train on 1718855 samples
1718855/1718855 [==============================] - 53s 31us/sample - loss: 0.0945


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1        395  0.49375   0.493750  0.493750
1        5        409  0.51125   0.102250  0.502461
2       10        426  0.53250   0.053250  0.509254
3       15        442  0.55250   0.036833  0.514497
4       20        456  0.57000   0.028500  0.518617]
Epoch: 1
Train on 1718855 samples
1718855/1718855 [==============================] - 53s 31us/sample - loss: 0.0366


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1        395  0.49375   0.493750  0.493750
1        5        409  0.51125   0.102250  0.502689
2       10        426  0.53250   0.053250  0.509412
3       15        438  0.54750   0.036500  0.513301
4       20        453  0.56625   0.028312  0.517809]
Epoch: 2
Train on 1718855 samples
1718855/1718855 [==============================] - 53s 31us/sample - loss: 0.0269


...

[   rank_at  hitcounts   recall  precision      ndcg
0        1        398  0.49750   0.497500  0.497500
1        5        422  0.52750   0.105500  0.512091
2       10        438  0.54750   0.054750  0.518496
3       15        452  0.56500   0.037667  0.523120
4       20        471  0.58875   0.029438  0.528691]
Epoch: 18
Train on 1718855 samples
1718855/1718855 [==============================] - 53s 31us/sample - loss: 0.0063


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1        396  0.49500   0.495000  0.495000
1        5        417  0.52125   0.104250  0.508795
2       10        438  0.54750   0.054750  0.517135
3       15        455  0.56875   0.037917  0.522732
4       20        470  0.58750   0.029375  0.527191]
Epoch: 19
Train on 1718855 samples
1718855/1718855 [==============================] - 53s 31us/sample - loss: 0.0061


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1        397  0.49625   0.496250  0.496250
1        5        418  0.52250   0.104500  0.509554
2       10        438  0.54750   0.054750  0.517710
3       15        456  0.57000   0.038000  0.523633
4       20        469  0.58625   0.029312  0.527474]


## MLP

In [120]:
MLP_val_metrics = NCF.train_model('MLP', samples, train_set, val_set.iloc[:800], store_path=f'../weights/MLP_weights_{run_num}/MLP_weights') 


Fitting MLP with parameters:
learning_rate                                             0.001
batch_size                                                  256
layers                                          [16, 32, 16, 8]
reg_layers                                    [0.0001, 0, 0, 0]
epochs                                                       20
sample_size                                              343771
num_neg                                                       4
ckpt_dir         ../NeuMF_storage/MLP_ckpts_am_40k_nolf_8/ckpts
optimizer                                                  Adam
Name: 0, dtype: object
Epoch: 0
Train on 1718855 samples


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1718855/1718855 [==============================] - 56s 33us/sample - loss: 0.2354


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1         35  0.04375   0.043750  0.043750
1        5        129  0.16125   0.032250  0.101947
2       10        197  0.24625   0.024625  0.129401
3       15        268  0.33500   0.022333  0.152736
4       20        342  0.42750   0.021375  0.174625]
Epoch: 1
Train on 1718855 samples
1718855/1718855 [==============================] - 56s 32us/sample - loss: 0.0786


...

[   rank_at  hitcounts   recall  precision      ndcg
0        1         29  0.03625   0.036250  0.036250
1        5        107  0.13375   0.026750  0.085028
2       10        184  0.23000   0.023000  0.115937
3       15        246  0.30750   0.020500  0.136496
4       20        305  0.38125   0.019062  0.153913]
Epoch: 18
Train on 1718855 samples
1718855/1718855 [==============================] - 158s 92us/sample - loss: 0.0486


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1         28  0.03500    0.03500  0.035000
1        5         95  0.11875    0.02375  0.074851
2       10        182  0.22750    0.02275  0.109212
3       15        267  0.33375    0.02225  0.137155
4       20        344  0.43000    0.02150  0.159838]
Epoch: 19
Train on 1718855 samples
1718855/1718855 [==============================] - 157s 92us/sample - loss: 0.0471


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1         37  0.04625   0.046250  0.046250
1        5         93  0.11625   0.023250  0.082224
2       10        175  0.21875   0.021875  0.115187
3       15        245  0.30625   0.020417  0.138134
4       20        303  0.37875   0.018937  0.155215]


## NeuMF

In [ ]:
NeuMF_val_metrics = NCF.train_model('NeuMF', samples, train_set, val_set.iloc[:800], store_path=f'../weights/NeuMF_weights_{run_num}/NeuMF_weights') 


Fitting NeuMF with parameters:
learning_rate                                               0.001
batch_size                                                    256
layers                                            [16, 32, 16, 8]
reg_layers                                      [0.0001, 0, 0, 0]
reg_mf                                             [1e-06, 1e-06]
nolf                                                            8
epochs                                                         20
sample_size                                                343771
num_neg                                                         4
ckpt_dir         ../NeuMF_storage/NeuMF_ckpts_am_40k_nolf_8/ckpts
optimizer                                                    Adam
Name: 0, dtype: object
Epoch: 0
Train on 1718855 samples


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1718855/1718855 [==============================] - 218s 127us/sample - loss: 0.2012


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1         38  0.04750   0.047500  0.047500
1        5        123  0.15375   0.030750  0.100177
2       10        201  0.25125   0.025125  0.131248
3       15        273  0.34125   0.022750  0.154767
4       20        338  0.42250   0.021125  0.173936]
Epoch: 1
Train on 1718855 samples
1718855/1718855 [==============================] - 212s 124us/sample - loss: 0.0493


...

[   rank_at  hitcounts   recall  precision      ndcg
0        1         24  0.03000   0.030000  0.030000
1        5        103  0.12875   0.025750  0.079123
2       10        177  0.22125   0.022125  0.108747
3       15        253  0.31625   0.021083  0.133942
4       20        333  0.41625   0.020812  0.157545]
Epoch: 16
Train on 1718855 samples
1718855/1718855 [==============================] - 214s 124us/sample - loss: 0.0202


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1         25  0.03125   0.031250  0.031250
1        5        111  0.13875   0.027750  0.083637
2       10        173  0.21625   0.021625  0.108563
3       15        245  0.30625   0.020417  0.132170
4       20        323  0.40375   0.020187  0.155187]
Epoch: 17
Train on 1718855 samples
1718855/1718855 [==============================] - 213s 124us/sample - loss: 0.0195


100% |########################################################################|


[   rank_at  hitcounts   recall  precision      ndcg
0        1         27  0.03375   0.033750  0.033750
1        5         90  0.11250   0.022500  0.072920
2       10        180  0.22500   0.022500  0.108970
3       15        270  0.33750   0.022500  0.138457
4       20        329  0.41125   0.020562  0.155878]
Epoch: 18
Train on 1718855 samples
1718855/1718855 [==============================] - 212s 124us/sample - loss: 0.0196


#### Load weights for NeuMF

In [ ]:
NCF.use_pretrain_model(GMF_weights_path=f'../weights/GMF_weights_{run_num}/GMF_weights',
                       MLP_weights_path=f'../weights/MLP_weights_{run_num}/MLP_weights')

In [ ]:
if __name__ == '__main__':
    val_metrics = train_model('NeuMF', train_set, test_set.iloc[:1000], store_path=f'../weights/NeuMF_weights_{run_num}/NeuMF_weights') 

---
## Evaluation

In [ ]:
steps = 5
rank_at = 20
sample_len = 100

In [72]:
NCF.NeuMF.load_weights(f'../weights/NeuMF_weights_{run_num}/NeuMF_weights').expect_partial()
NCF.GMF.load_weights(f'../weights/GMF_weights_{run_num}/GMF_weights').expect_partial()
NCF.MLP.load_weights(f'../weights/MLP_weights_{run_num}/MLP_weights').expect_partial()

---
## GMF

### Full set scores

In [ ]:
%%time
ranked_df_full = NCF.get_predictions('GMF', test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(ranked_df_full, steps, rank_at, stats=False)

### Sample scores

In [ ]:
%%time
ranked_df = NCF.sample_prediction('GMF', train_set, test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(ranked_df, steps, rank_at, stats=False)

---
## MLP

### Full set scores

In [ ]:
%%time
MLP_ranked_df_full = NCF.get_predictions('MLP', test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(MLP_ranked_df_full, steps, rank_at, stats=False)

### Sample scores

In [ ]:
%%time
MLP_ranked_df = NCF.sample_prediction('MLP', train_set, test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(MLP_ranked_df, steps, rank_at, stats=False)

In [83]:
test_set_og = test_set

In [90]:
test_set = test_set.iloc[:500]
sample_len = 15000
rank_at = 20

In [85]:
model, _ = NCF.get_model('MLP')
user_items = train_set.groupby('user_id')['item_id'].apply(list)
test_user_items = test_set.groupby('user_id')['item_id'].apply(list)
train_items = train_set.item_id.unique()

In [91]:
preds_ranked = []
true_items = []
pbar = progressbar.ProgressBar()
for u in pbar(test_user_items.index):
    true_item = test_user_items[u]
    pos_items = user_items[u]
    neg_items = set(train_items) - set(pos_items)
    neg_sample = np.random.choice(list(neg_items), sample_len-1)
    total_sample = np.append(neg_sample, true_item)
    user_array = np.full(len(total_sample), u, dtype='int32')

    preds = np.hstack(model.predict([user_array, total_sample], batch_size=sample_len, verbose=0))
    print(np.sort(preds)[-20:][::-1])
#     best = total_sample[np.argsort(preds)][-rank_at:][::-1]
    ids = np.argpartition(preds, -rank_at)[-rank_at:]
    best_ids = np.argsort(preds[ids])[::-1]
    best = total_sample[ids[best_ids]]

    preds_ranked.append(best)
    true_items.append(true_item)

ranked_df = pd.DataFrame(list(zip(test_user_items.index, preds_ranked, true_items)),
                         columns=['users', 'pred_items_ranked', 'true_id'])

[0.99993694 0.9997149  0.9997149  0.99930465 0.99930465 0.99930465
 0.99922156 0.9991722  0.9991722  0.9989722  0.9989722  0.99892175
 0.99891925 0.9985071  0.99801624 0.9976553  0.99749786 0.99709713
 0.99707925 0.99697423]
[0.9999896  0.9999553  0.9999374  0.9999374  0.9999144  0.99990296
 0.99989116 0.999884   0.9998658  0.9998234  0.9998091  0.999778
 0.999778   0.9996866  0.9996866  0.9996866  0.9994733  0.9994704
 0.9994173  0.9994173 ]
[0.99992776 0.99992776 0.99986124 0.99986124 0.99986124 0.99963975
 0.9994522  0.9994522  0.999421   0.9993412  0.9993412  0.9993382
 0.9993266  0.9991103  0.9991103  0.99906296 0.99906296 0.9989699
 0.99893326 0.9985562 ]


[0.99998736 0.99998736 0.99998736 0.9999825  0.99997747 0.99997747
 0.9999378  0.9998528  0.9998528  0.9997563  0.9997563  0.9997383
 0.99973387 0.99960726 0.9994704  0.9994704  0.99932146 0.99904233
 0.99904233 0.99891686]
[0.99986863 0.999806   0.99860734 0.9982439  0.9981907  0.9979017
 0.9978877  0.99786854 0.99721044 0.99710053 0.9950389  0.99476516
 0.9942932  0.9938061  0.9925027  0.9923254  0.9922078  0.9921902
 0.99216133 0.99111253]
[0.99998546 0.99996555 0.99996555 0.999948   0.9998946  0.99986565
 0.99973637 0.9994678  0.9993253  0.9992513  0.9992513  0.99895084
 0.9989196  0.9989196  0.99862003 0.99848914 0.99848914 0.99847335
 0.998243   0.9982242 ]
[0.9999206  0.9997069  0.9996755  0.99935824 0.9990368  0.9990368
 0.9986858  0.9986858  0.9984206  0.9979334  0.9974577  0.9973883
 0.9973883  0.99737096 0.9971973  0.99690515 0.9962075  0.99616784
 0.9956369  0.9955005 ]
[0.9998617  0.99950325 0.99931777 0.9992817  0.9990682  0.9990682
 0.9987016  0.9987016  0.9986506  0.998

[0.9999887  0.9999852  0.9999852  0.9999827  0.9999827  0.999972
 0.9999515  0.9999515  0.9999411  0.99985826 0.9998565  0.9996896
 0.9996896  0.99966383 0.99966383 0.99966383 0.9996537  0.99950314
 0.9993832  0.9993832 ]
[0.9999933  0.9999143  0.99983585 0.9997588  0.9997063  0.99968064
 0.9996037  0.9996037  0.9996037  0.99954444 0.99950445 0.9993117
 0.99929726 0.99929726 0.9992331  0.9992331  0.9990476  0.9989587
 0.99882275 0.9987387 ]
[0.9999894  0.99992657 0.99992657 0.9994165  0.9994165  0.99920386
 0.998607   0.99859506 0.99854374 0.9982352  0.9981951  0.9981951
 0.99818605 0.998159   0.998159   0.9980934  0.9978992  0.9976432
 0.9976084  0.9975242 ]
[0.99999046 0.9999838  0.9999019  0.9998036  0.9995535  0.9995535
 0.9994696  0.9993918  0.999281   0.99920017 0.9991485  0.9988961
 0.9988961  0.9985098  0.99847025 0.99847025 0.99847025 0.9982999
 0.9981918  0.9981918 ]
[0.99994826 0.99992025 0.9998375  0.9996922  0.9996922  0.99966717
 0.999635   0.999635   0.99961334 0.9992845

[0.99998784 0.99984443 0.999816   0.9997112  0.9994748  0.9994493
 0.9994178  0.99933416 0.99925214 0.99925214 0.99852175 0.99814546
 0.9979207  0.9979207  0.9979207  0.997623   0.997623   0.9974813
 0.9968317  0.99606776]
[0.99999034 0.99999034 0.99998593 0.999982   0.9999727  0.9999701
 0.99995553 0.9999453  0.9999403  0.999926   0.999926   0.999926
 0.99991274 0.9998833  0.9998547  0.99978155 0.99974436 0.9997143
 0.9995913  0.999413  ]
[0.99996865 0.9998031  0.9998018  0.99978524 0.9995528  0.9994462
 0.9994462  0.9994462  0.999321   0.99916685 0.99910057 0.9987955
 0.99872655 0.9987103  0.9986265  0.99848574 0.99848574 0.99831235
 0.9981067  0.9981067 ]


[0.99973446 0.9994167  0.99909914 0.99904805 0.99904805 0.99904805
 0.99904805 0.99886703 0.99886703 0.99879515 0.9987735  0.9987112
 0.9987112  0.998623   0.998623   0.998623   0.99859554 0.9985123
 0.99849975 0.9984635 ]
[0.9996387  0.9996387  0.9996288  0.9996288  0.99936825 0.99922156
 0.99922156 0.99891424 0.9985067  0.9982103  0.9982103  0.9976452
 0.9976197  0.9975666  0.9975666  0.9974935  0.99745506 0.99745506
 0.99707055 0.99704677]
[0.99994767 0.999846   0.9988642  0.99856645 0.9983016  0.9981652
 0.99788123 0.99767405 0.9974011  0.9973496  0.9973496  0.9971246
 0.9971246  0.996797   0.996797   0.99673826 0.99650675 0.9964857
 0.9964857  0.9963497 ]
[0.99965525 0.9995284  0.9995284  0.9995284  0.9994702  0.9993587
 0.9993088  0.9993088  0.99919397 0.9991774  0.9991774  0.99846387
 0.99825555 0.9981863  0.9977835  0.997577   0.997577   0.997577
 0.9974222  0.9974185 ]
[0.9999963  0.99997675 0.9998016  0.9998016  0.9997427  0.999701
 0.9996835  0.99940526 0.99940526 0.99940526

...

[0.99966455 0.99907804 0.99878675 0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99962175 0.99936014 0.99924266 0.99924266 0.9987741
 0.9987692  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99924266 0.99924266 0.99907804 0.99878675 0.99878675 0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99967    0.99962664 0.99962175 0.99928135 0.99907804 0.9987741
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99907804 0.99907804 0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0

[0.9998702  0.9998702  0.99962175 0.99924266 0.99907804 0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99967    0.99967    0.99967    0.99962664 0.99962175
 0.99928135 0.99878675 0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99955684 0.99955684 0.99936014 0.99924266 0.99907804
 0.9988337  0.9988337  0.9987692  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99955684 0.99936014 0.99907804 0.9988337  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99967    0.99967    0.99966455 0.99962664 0.99928135 0.99878675
 0.99878675 0.9987355  0.9987355  0.9987355

[0.99967    0.99928135 0.99907804 0.99878675 0.9987692  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99966455 0.99962664 0.99962664 0.99955684 0.99928135 0.99928135
 0.99924266 0.99907804 0.99907804 0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99962664 0.99962664 0.99936014 0.99928135 0.99924266
 0.99924266 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]


[0.99967    0.99967    0.99966455 0.99962175 0.99962175 0.99962175
 0.99928135 0.99928135 0.99878675 0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99967    0.99966455 0.99966455 0.99966455 0.99962664 0.99962664
 0.99955684 0.99955684 0.99955684 0.99936014 0.99936014 0.99928135
 0.99924266 0.99924266 0.99907804 0.9988337  0.9988337  0.99878675
 0.99878675 0.9987692 ]
[0.99962175 0.99936014 0.99928135 0.99928135 0.99924266 0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.9998702  0.99967    0.99962664 0.99936014 0.99926955 0.99924266
 0.99924266 0.9988337  0.9988337  0.99878675 0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99967    0.99962664 0.99955684 0.99955684 0.99936014 0.99936014
 0.99924266 0.99907804 0.9988337  0.9988

[0.9998702  0.99967    0.99955684 0.99955684 0.99936014 0.99928135
 0.99928135 0.99907804 0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355  0.9987355  0.9987355  0.9987355  0.9987355
 0.9987355  0.9987355 ]
[0.99962664 0.99962175 0.99962175 0.99962175 0.99955684 0.99928135
 0.99928135 0.99928135 0.99926955 0.99924266 0.99924266 0.99907804
 0.99907804 0.9988337  0.9988337  0.99878675 0.99878675 0.9987741
 0.9987355  0.9987355 ]


In [92]:
get_metrics(ranked_df, 5, rank_at, stats=False)

,rank_at,hitcounts,recall,precision,ndcg
0,1,0,0.000,0.0000,0.000000
1,5,5,0.010,0.0020,0.003956
2,10,24,0.048,0.0048,0.015909
3,15,51,0.102,0.0068,0.030284
4,20,68,0.136,0.0068,0.038184


---
## NeuMF

### Full set scores

In [330]:
test_set_og = test_set

In [336]:
test_set = test_set_og

In [331]:
test_set = test_set.iloc[:100]

In [340]:
sample_len = 10000

In [310]:
u = test_set.user_id.unique()[0]
true_item = test_user_items[u]
pos_items = user_items[u]
neg_items = set(train_items) - set(pos_items)
neg_sample = np.random.choice(list(neg_items), sample_len-1)
total_sample = np.append(neg_sample, true_item)
user_array = np.full(len(total_sample), u, dtype='int32')

In [342]:
preds = np.hstack(model.predict([user_array, total_sample], batch_size=sample_len, verbose=0))

In [344]:
total_sample[np.argsort(preds)[-2:][::-1]]

array([  417, 10747])

In [345]:
ids = np.argpartition(preds, -2)[-2:]
best_ids = np.argsort(preds[ids])[::-1]
best = total_sample[ids[best_ids]]
best

array([ 417, 3868])

array([1, 0])

In [238]:
a = np.array([1,2,3,4,10,15,16])
ids = np.argpartition(a, -2)[-2:]
best_ids = np.argsort(a[ids])[::-1]
best = a[ids[best_ids]]
best

array([16, 15])

In [239]:
a[np.argsort(a)[-2:][::-1]]

array([16, 15])

In [234]:
best

array([15, 16])

In [231]:
a

array([ 1,  2,  3,  4, 10, 15, 16])

In [334]:
get_metrics(ranked_df, steps, rank_at, stats=False)

,rank_at,hitcounts,recall,precision,ndcg
0,1,100,1.0,1.000000,1.0
1,5,100,1.0,0.200000,1.0
2,10,100,1.0,0.100000,1.0
3,15,100,1.0,0.066667,1.0
4,20,100,1.0,0.050000,1.0


In [220]:
get_metrics(ranked_df, steps, rank_at, stats=False)

,rank_at,hitcounts,recall,precision,ndcg
0,1,2,0.02,0.020,0.020000
1,5,22,0.22,0.044,0.127003
2,10,29,0.29,0.029,0.149119
3,15,36,0.36,0.024,0.167483
4,20,38,0.38,0.019,0.172113


In [215]:
get_metrics(ranked_df, steps, rank_at, stats=False)

,rank_at,hitcounts,recall,precision,ndcg
0,1,776,0.047742,0.047742,0.047742
1,5,2614,0.160822,0.032164,0.104654
2,10,3817,0.234835,0.023483,0.128561
3,15,4373,0.269041,0.017936,0.137625
4,20,4799,0.295250,0.014763,0.143807


In [33]:
len(train_set)

343771

In [ ]:
%%time
NeuMF_ranked_df_full = NCF.get_predictions('NeuMF', test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(NeuMF_ranked_df_full, steps, rank_at, stats=False)

In [40]:
metrics_all_sample.to_pickle('../results/NCF/NCF_metrics_all_sample_am_40k')

### Sample scores

In [ ]:
%%time
NeuMF_ranked_df = NCF.sample_prediction('NeuMF', train_set, test_set)

In [ ]:
from Evaluation import get_metrics
get_metrics(NeuMF_ranked_df, steps, rank_at, stats=False)

In [87]:
%%time
from Evaluation import get_metrics
test_set = test_set.iloc[:300]
sample_sizes = [20, 50, 100, 200, 400, 10000]
metrics = []
for sample_size in sample_sizes:
    ranked_sample_df = NCF.sample_prediction('NeuMF', train_set, test_set, sample_len=sample_size)
    metric = get_metrics(ranked_sample_df, steps, rank_at)
    metrics.append(metric)
#     get_metrics(ranked_sample_df).to_pickle(f'../results/NCF/NCF_metrics_{sample_size}_sample_am_40k')

100% |########################################################################|


Obtaining metrics time: 0.33
   rank_at  hitcounts    recall  precision      ndcg
0        1         14  0.046667   0.046667  0.046667
1        5         47  0.156667   0.031333  0.099743
2       10        100  0.333333   0.033333  0.155061
3       15        171  0.570000   0.038000  0.216693
4       20        300  1.000000   0.050000  0.317509


100% |########################################################################|


Obtaining metrics time: 0.34
   rank_at  hitcounts    recall  precision      ndcg
0        1          6  0.020000   0.020000  0.020000
1        5         22  0.073333   0.014667  0.047401
2       10         36  0.120000   0.012000  0.062457
3       15         50  0.166667   0.011111  0.074852
4       20         66  0.220000   0.011000  0.087413


KeyboardInterrupt: 

In [58]:
metrics

[   rank_at  hitcounts    recall  precision      ndcg
 0        1         50  0.166667   0.166667  0.166667
 1        5        124  0.413333   0.082667  0.283302
 2       10        198  0.660000   0.066000  0.361850
 3       15        253  0.843333   0.056222  0.410768
 4       20        300  1.000000   0.050000  0.447358,
    rank_at  hitcounts    recall  precision      ndcg
 0        1         45  0.150000   0.150000  0.150000
 1        5         60  0.200000   0.040000  0.172079
 2       10        105  0.350000   0.035000  0.220121
 3       15        143  0.476667   0.031778  0.253404
 4       20        181  0.603333   0.030167  0.283313,
    rank_at  hitcounts    recall  precision      ndcg
 0        1         45  0.150000   0.150000  0.150000
 1        5         48  0.160000   0.032000  0.154246
 2       10         63  0.210000   0.021000  0.169534
 3       15         82  0.273333   0.018222  0.186272
 4       20        104  0.346667   0.017333  0.203395,
    rank_at  hitcounts   

In [ ]:
get_metrics(ranked_df_full, steps, rank_at).to_pickle('../results/NCF/NCF_metrics_ml_1m')
get_metrics(ranked_df, steps, rank_at).to_pickle('../results/NCF/NCF_metrics_100_sample_ml_1m')

---
### Multiprocessing for multiple samples creatings 

In [ ]:
# from progressbar import progressbar
# def create_samples(params, data, name=''):
#         print(f'Creating Samples for {name}')
# #         _, params = self.get_model(name)
#         all_user_inputs, all_item_inputs, all_labels = [], [], []
#         user_items = data.groupby('user_id')['item_id'].apply(list)
#         train_users = data.user_id.unique()
#         train_items = data.item_id.unique()

#         pbar = progressbar.ProgressBar()
#         for n in pbar(range(params['epochs'])):
#             user_inputs, item_inputs, labels = [], [], []
#             for s in range(int(params['sample_size'])):
#                 # Add positive item
#                 u = np.random.choice(train_users)
#                 u_items = user_items[u]
#                 i = np.random.choice(u_items)

#                 user_inputs.append(u)
#                 item_inputs.append(i)
#                 labels.append(1)

#                 # Add negative item
#                 for i in range(params['num_neg']):
#                     j = np.random.choice(train_items)
#                     while j in u_items:  # neg item j cannot be in the set of pos items of user u
#                         j = np.random.choice(train_items)

#                     user_inputs.append(u)
#                     item_inputs.append(j)
#                     labels.append(0)

#             all_user_inputs.append(user_inputs)
#             all_item_inputs.append(item_inputs)
#             all_labels.append(labels)

#         return [all_user_inputs, all_item_inputs, all_labels]

In [ ]:
# import multiprocessing as mp
# if __name__ == '__main__':
    
#     with mp.Pool(processes=2) as pool:
#         results = pool.starmap(create_samples, [(GMF_params, train_set, 'GMF'), (MLP_params, train_set, 'MLP')])
        

## MP sampler

In [ ]:
def subsampler(worker_id, epochs, user_items, train_users, train_items, params):
    print(f'worker {workes_id} started')
    all_user_inputs, all_item_inputs, all_labels = [], [], []
    for epoch in range(epochs): 
        user_inputs, item_inputs, labels = [], [], []
        for s in range(params['sample_size']):
            # Add positive item
            u = np.random.choice(train_users)
            u_items = user_items[u]
            i = np.random.choice(u_items)

            user_inputs.append(u)
            item_inputs.append(i)
            labels.append(1)

            # Add negative item
            for i in range(params['num_neg']):
                j = np.random.choice(train_items)
                while j in u_items:  # neg item j cannot be in the set of pos items of user u
                    j = np.random.choice(train_items)

                user_inputs.append(u)
                item_inputs.append(j)
                labels.append(0)
                
        all_user_inputs.append(user_inputs)
        all_item_inputs.append(item_inputs)
        all_labels.append(labels)
        
    return {'u':all_user_inputs, 'i':all_item_inputs, 'l':all_labels}

In [ ]:
%%time
import multiprocessing as mp
from progressbar import progressbar
if __name__ == '__main__':
    processors = mp.cpu_count()
    params = NCF.GMF_params
#     params = NCF.MLP_params
#     params = NCF.NeuMF_params
    data = train_set

    all_user_inputs, all_item_inputs, all_labels = [], [], []
    user_items = data.groupby('user_id')['item_id'].apply(list)
    train_users = data.user_id.unique()
    train_items = data.item_id.unique()
    
    epoch_splits = np.array_split(np.array(range(params['epochs'])), processors)
    args = []
    for worker_id, epoch_split in enumerate(epoch_splits):
        args.append((worker_id, len(epoch_split), user_items, train_users, train_items, params))
        
    with mp.Pool(processes=processors) as pool:
        results = pool.starmap(subsampler, args)
    
    all_user_inputs, all_item_inputs, all_labels = [], [], []
    samples = [[], [], []]
    for res_epochs in results:
        all_user_inputs.extend(res_epochs['u'])
        all_item_inputs.extend(res_epochs['i'])
        all_labels.extend(res_epochs['l'])
    
    GMF_samples_mp = [all_user_inputs, all_item_inputs, all_labels]
#     MLP_samples_mp = [all_user_inputs, all_item_inputs, all_labels]
#     NeuMF_samples_mp = [all_user_inputs, all_item_inputs, all_labels]


In [ ]:
import multiprocessing as mp
from Evaluation import get_metrics
def train_model(name='', train_set=[], val_set=[], verbose=1, store_path=''):
    model, params = NCF.get_model(name)

    ckpts_prefix = os.path.join(params['ckpt_dir'], "ckpt")
    ckpts_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_prefix,    
                                                     monitor='loss',    
                                                     mode='min',    
                                                     save_best_only=True,
                                                     save_weights_only=True)

    if len(train_set) == 0:
        raise Exception('No samples available, create samples first using: create_samples')
        
    val_metrics = fit(model, params, train_set, val_set, [ckpts_callback], verbose)

    if len(store_path) > 0:
        model.save_weights(store_path)
        
    return val_metrics


def fit(model, params, train_set, val_set, callbacks, verbose):
    print(f'\nFitting {model._name} with parameters:')
    print(pd.DataFrame.from_dict(params, orient='index'))
    user_items = train_set.groupby('user_id')['item_id'].apply(list)
    train_users = train_set.user_id.unique()
    train_items = train_set.item_id.unique()
    num_processes = mp.cpu_count()
    val_metrics = []
    for epoch in range(params['epochs']):
        print(f'Epoch: {epoch}')
        user_inputs, item_inputs, labels = create_sample(user_items, train_users, train_items, params, num_processes)
        hist = model.fit([np.array(user_inputs), np.array(item_inputs)], 
                  np.array(labels), 
                  batch_size=params['batch_size'], 
                  verbose=verbose, 
                  epochs=1, 
                  shuffle=True,
                  callbacks=callbacks)
        
        ranked_df = sample_prediction(model, train_set, val_set)
        val_metrics.append(get_metrics(ranked_df, 5, 20, stats=False))
        print(val_metrics[-1:])

        NCF.history[model._name]['loss'].append(round(hist.history['loss'][0],5))
    return val_metrics
        
        
def create_sample(user_items, train_users, train_items, params, num_processes):
    samples_sizes_split = np.array_split(np.array(range(params['sample_size'])),8)
    args = []
    for samples_size in samples_sizes_split:
        args.append((user_items, train_users, train_items, len(samples_size), params['num_neg']))
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(create_sample_worker, args)

    user_inputs, item_inputs, labels = [], [], []
    for res_epochs in results: 
        user_inputs.extend(res_epochs['u'])
        item_inputs.extend(res_epochs['i'])
        labels.extend(res_epochs['l'])

    return user_inputs, item_inputs, labels


def create_sample_worker(user_items, train_users, train_items, sample_size, num_neg):
    user_inputs, item_inputs, labels = [], [], []
#     print('worker started')
    for s in range(sample_size):
        # Add positive item
        u = np.random.choice(train_users)
        u_items = user_items[u]
        i = np.random.choice(u_items)

        user_inputs.append(u)
        item_inputs.append(i)
        labels.append(1)

        # Add negative item
        for i in range(num_neg):
            j = np.random.choice(train_items)
            while j in u_items:  # neg item j cannot be in the set of pos items of user u
                j = np.random.choice(train_items)

            user_inputs.append(u)
            item_inputs.append(j)
            labels.append(0)

    return {'u':user_inputs, 'i':item_inputs, 'l':labels}